# Vectorizing text

As you may expect it, the goal of vectorizing text is to transform text input into a numerical representation. Ideally, this numerical representation should carry semantic meaning that make it easier for an ML model to detect patterns in a corpus of text.

There are multiple ways to vectorize text. I'll demonstrate two here:
- TF-IDF, which builds vectors for documents based on relative word frequency within the given corpus
- Pretrained models which leverage information from other corpuses

We start by loading the data, and getting a training and test split.

In [1]:
import pandas as pd
import spacy
import umap
import numpy as np 
from pathlib import Path
import sys
sys.path.append("..")

%matplotlib inline
%load_ext autoreload
%autoreload 2

from ml_editor.data_processing import format_raw_df, get_split_by_author

data_path = Path('../data/writers.csv')
df = pd.read_csv(data_path)
df = format_raw_df(df.copy())

train_author, test_author = get_split_by_author(df[df["is_question"]])

/Users/emmanuel.ameisen/ml_editor/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## TF-IDF on ngrams

[TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) creates embeddings based on the relative frequency of each word in each document as compared to the corpus as a whole. We create TF-IDF embeddings using sklearn below.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

questions = train_author[train_author["is_question"]]
raw_text = questions["body_text"]
# Extract a label to use as a color on our plots. 
# This label does not need to be the same label as the one for the classifier.
sent_labels = questions["AcceptedAnswerId"].notna()

sent_labels.value_counts()

True     2907
False    2769
Name: AcceptedAnswerId, dtype: int64

In [3]:
# Create an instance of a tfidf vectorizer, 
# We could use CountVectorizer for a non normalized version
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_features=2**21)

# Fit our vectorizer to questions in our dataset
# Returns an array of vectorized text
bag_of_words = vectorizer.fit_transform(raw_text)

bag_of_words.shape

(5676, 27381)

We have now vectorized our text. This process is also called embedding, and the resulting vectors are often referred to as embeddings. We can visualize the embeddings by projecting them to two dimensions using a dimensionality reducing technique such as PCA, t-SNE or [UMAP](https://umap-learn.readthedocs.io/en/latest/). We use UMAP here.

In [ ]:
umap_embedder = umap.UMAP()
umap_bow = umap_embedder.fit_transform(bag_of_words)

In [ ]:
from ml_editor.data_visualization import plot_embeddings

plot_embeddings(umap_bow, sent_labels)

This is a simplified representation of our data (the actual vectors are much larger than two dimensions), but can help discern trends, or with future labeling efforts. As you add features, it can be valuable to visualize embeddings to estimate whether classes look more separated.

## Pre-trained word embeddings

For this second approach, we load existing word vectors that were trained on a much larger corpus, which allows us to leverage general information from outside of our corpus. We will do so using the `spacy` library.

In [ ]:
# Load a large model, and disable pipeline unnecessary parts for our task
# This speeds up the vectorization process significantly
# See https://spacy.io/models/en#en_core_web_lg for details about the model
nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner", "textcat"])

In [ ]:
# Get the vector for each of our questions
# By default, the vector returned is the average of all vectors in the sentence
# See https://spacy.io/usage/vectors-similarity for more
spacy_emb = train_author[train_author["is_question"]]["body_text"].apply(lambda x: nlp(x).vector)
embeddings = np.vstack(spacy_emb)

In [ ]:
umap_embedder = umap.UMAP()
umap_emb = umap_embedder.fit_transform(embeddings)

Let's visualize the pretrained embeddings using UMAP

In [ ]:
plot_embeddings(umap_emb, sent_labels)

The topology of our dataset looks different because our embedding method is different. The underlying data however, is the same.

Different embedding methods will cause a dataset to be represented differently. When using embeddings trained only on your data, documents that use the same vocabulary will be embedded close to each other. Using models trained on other corpuses however allows you to leverage information from these corpuses. With these models, semantically similar sentences may be embedded close to each other even if their vocabulary is completely different.

Changing the way you vectorize your data can often have a significant impact on model performance.